In [ ]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Create the feature matrix and the target vector
features = ["rank", "personName", "age", "category", "country",
          "gender_F", "gender_M"]

target = ["finalWorth"]


In [ ]:
# Load the data
df = pd.read_csv('../Billionaire_Analysis/Resources/cleaned_billionaire.csv')
df.head()


In [ ]:
# Create X and y variables
# Create our features
X = df.loc[:, features].copy()

# Create our target
## This calls the target variable from ln[2] 
## and will locate and copy all values within the "finalWorth" column
y = df.loc[:, target].copy()


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
# Create a model object
from sklearn.linear_model import LinearRegression
model = LinearRegression()


In [ ]:
# Fit the model
model.fit(X_train, y_train)


In [ ]:
# Predict with the model
y_pred = model.predict(X_test)


In [ ]:
# Prediction DataFrame
pd.DataFrame({"Prediction": y_pred, "Actual": y_test})


In [ ]:
# Plot the prediction
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, color="green")
plt.show()


In [ ]:
# Evaluate the model on the test data
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)